<a href="https://colab.research.google.com/github/Bayrem-ben/Bayrem-ben.github.io/blob/master/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn.model_selection import train_test_split

df = dataset.copy()
df['sex'] = df['sex'].replace({'female': 0, 'male': 1})
df['smoker'] = df['smoker'].replace({'no': 0, 'yes': 1})

x = pd.get_dummies(df, columns=['region'], drop_first=True)
y = df['expenses']

train_dataset, test_dataset, train_labels, test_labels = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
CATEGORICAL_COLUMNS = ['age', 'bmi','sex','smoker', 'children']
NUMERIC_COLUMNS = ['expenses']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = df_dummies[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in df_pop:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(train_dataset, train_labels)
eval_input_fn = make_input_fn(test_dataset, test_labels, num_epochs=1, shuffle=False)

classifier = tf.estimator.LinearRegressor(
    feature_columns=df,
    optimizer=tf.keras.optimizers.Ftrl(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))
classifier.train(train_input_fn)
result = classifier.evaluate(eval_input_fn)

print(result)

In [ ]:

model = tf.estimator.LinearClassifier(
    feature_columns=df_dummies, optimizer=tf.keras.optimizers.Ftrl(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))
""" 
, hidden_units=[30,10], n_classes=3   
    model_dir=None, n_classes=2, weight_column=None,
    label_vocabulary=None, optimizer='Ftrl', config=None,
    warm_start_from=None,
    loss_reduction=losses_utils.ReductionV2.SUM_OVER_BATCH_SIZE,
    sparse_combiner='sum'
)


model.compile(
    optimizer=keras.optimizers.RMSprop(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
model.summary()
"""

In [ ]:
def f_input_fn(features, labels, training=True, batch_size= 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    dataset = dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)
    
classifier = tf.estimator.LinearRegressor(
    feature_columns=df_dummies,
    optimizer=tf.keras.optimizers.Ftrl(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))
    
classifier.train(input_fn=lambda : f_input_fn(train_dataset, train_labels, training=True), steps=1000)

classifier.evaluate(input_fn=lambda : f_input_fn(test_dataset, test_labels, 
                                      training=False),
                                      steps=1000)

In [ ]:
""" another methode but no evaluate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
regressor = LinearRegression()

regressor.fit(train_dataset, train_labels)
y_prediction = regressor.predict(test_dataset)

accuracy_score(y_true = test_labels.astype(int), y_pred = y_prediction.astype(int))
train_s = regressor.score(train_dataset, train_labels)
test_s = regressor.score(test_dataset, test_labels)
print(train_s, test_s)
loss = regressor.evaluate(test_dataset, test_labels)
"""

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
